In [ ]:

import pandas as pd
import plotly.express as px
from ipywidgets import widgets, interactive, VBox
import datetime

csv_file_path = 'Palma Bay Complete 2025_2.csv'
timestamps = []
tidal_heights = []
with open(csv_file_path, 'r') as file:
    for line in file:
        parts = line.strip().split(',')
        if len(parts) == 3:  # Ensure correct data format
            datetime_str, time_str, height_str = parts
            datetime_val = pd.to_datetime(datetime_str + ' ' + time_str, format='%d-%m-%Y %H:%M')
            tidal_height = float(height_str.replace('m.cm', ''))
            timestamps.append(datetime_val)
            tidal_heights.append(tidal_height)

# Create DataFrame from the loaded data
df = pd.DataFrame({'Timestamp': timestamps, 'Tidal_Height(m)': tidal_heights})
df['Date'] = pd.to_datetime(df['Timestamp']).dt.floor('d')  # Normalize time to the date for better aggregation

# Function to update the plot based on user interaction
def update_plot(start_date, end_date, height_range):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    height_min, height_max = height_range  # Unpack the range tuple into min and max
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date) &
                     (df['Tidal_Height(m)'] >= height_min) & (df['Tidal_Height(m)'] <= height_max)]
    
    # Check if data is available to plot
    if not filtered_df.empty:
        fig = px.bar(filtered_df, x='Date', y='Tidal_Height(m)', title='Tidal Heights Over Selected Dates')
        fig.update_layout(xaxis_title='Date', yaxis_title='Tidal Height (meters)', xaxis=dict(tickformat="%Y-%m-%d"))
        fig.show()
    else:
        print("No data matches the criteria.")

# Widgets for interactivity
start_date_picker = widgets.DatePicker(description='Start Date', value=pd.to_datetime(df['Date'].min()))
end_date_picker = widgets.DatePicker(description='End Date', value=pd.to_datetime(df['Date'].max()))
height_range_slider = widgets.FloatRangeSlider(
    value=[0, 2.4],
    min=0,
    max=10,
    step=0.1,
    description='Height Range:',
    continuous_update=False
)

# Display the widgets and output interactively
out = widgets.interactive_output(update_plot, {'start_date': start_date_picker, 'end_date': end_date_picker, 'height_range': height_range_slider})
display(VBox([start_date_picker, end_date_picker, height_range_slider, out]))
